In [1]:
import numpy as np
from scipy.sparse import csr_matrix
#import assembler
from io import StringIO
from scipy.io import mmread
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import lgmres
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import minres
import scipy
from numpy.linalg import solve
from matplotlib import pyplot as plt
from scipy.sparse.linalg import LinearOperator, spilu
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import bicgstab
import krypy
import csv
import time
from skfem import *
from skfem.models.poisson import unit_load

In [2]:
Problem='Ex02'
#Kirchhoff plate bending problem
r"""

This example demonstrates the solution of a slightly more complicated problem
with multiple boundary conditions and a fourth-order differential operator. We
consider the `Kirchhoff plate bending problem
<https://en.wikipedia.org/wiki/Kirchhoff%E2%80%93Love_plate_theory>`_ which
finds its applications in solid mechanics. For a stationary plate of constant
thickness :math:`d`, the governing equation reads: find the deflection :math:`u
: \Omega \rightarrow \mathbb{R}` that satisfies

.. math::
    \frac{Ed^3}{12(1-\nu^2)} \Delta^2 u = f \quad \text{in $\Omega$},
where :math:`\Omega = (0,1)^2`, :math:`f` is a perpendicular force,
:math:`E` and :math:`\nu` are material parameters.
In this example, we analyse a :math:`1\,\text{m}^2` plate of steel with thickness :math:`d=0.1\,\text{m}`.
The Young's modulus of steel is :math:`E = 200 \cdot 10^9\,\text{Pa}` and Poisson
ratio :math:`\nu = 0.3`.

In reality, the operator

.. math::
    \frac{Ed^3}{12(1-\nu^2)} \Delta^2 
is a combination of multiple first-order operators:

.. math::
    \boldsymbol{K}(u) = - \boldsymbol{\varepsilon}(\nabla u), \quad \boldsymbol{\varepsilon}(\boldsymbol{w}) = \frac12(\nabla \boldsymbol{w} + \nabla \boldsymbol{w}^T),
.. math::
    \boldsymbol{M}(u) = \frac{d^3}{12} \mathbb{C} \boldsymbol{K}(u), \quad \mathbb{C} \boldsymbol{T} = \frac{E}{1+\nu}\left( \boldsymbol{T} + \frac{\nu}{1-\nu}(\text{tr}\,\boldsymbol{T})\boldsymbol{I}\right),
where :math:`\boldsymbol{I}` is the identity matrix. In particular,

.. math::
    \frac{Ed^3}{12(1-\nu^2)} \Delta^2 u = - \text{div}\,\textbf{div}\,\boldsymbol{M}(u).
There are several boundary conditions that the problem can take.
The *fully clamped* boundary condition reads

.. math::
    u = \frac{\partial u}{\partial \boldsymbol{n}} = 0,
where :math:`\boldsymbol{n}` is the outward normal.
Moreover, the *simply supported* boundary condition reads

.. math::
    u = 0, \quad M_{nn}(u)=0,
where :math:`M_{nn} = \boldsymbol{n} \cdot (\boldsymbol{M} \boldsymbol{n})`.
Finally, the *free* boundary condition reads

.. math::
    M_{nn}(u)=0, \quad V_{n}(u)=0,
where :math:`V_n` is the `Kirchhoff shear force <https://arxiv.org/pdf/1707.08396.pdf>`_. The exact
definition is not needed here as this boundary condition is a
natural one.

The correct weak formulation for the problem is: find :math:`u \in V` such that

.. math::
    \int_\Omega \boldsymbol{M}(u) : \boldsymbol{K}(v) \,\mathrm{d}x = \int_\Omega fv \,\mathrm{d}x \quad \forall v \in V,
where :math:`V` is now a subspace of :math:`H^2` with the essential boundary
conditions for :math:`u` and :math:`\frac{\partial u}{\partial \boldsymbol{n}}`.

Instead of constructing a subspace for :math:`H^2`, we discretise the problem
using the `non-conforming Morley finite element
<https://users.aalto.fi/~jakke74/WebFiles/Slides-Niiranen-ADMOS-09.pdf>`_ which
is a piecewise quadratic :math:`C^0`-continuous element for biharmonic problems.

The full source code of the example reads as follows:

.. literalinclude:: examples/ex02.py
    :start-after: EOF"""
from skfem import *
from skfem.models.poisson import unit_load
import numpy as np

m = (
    MeshTri.init_symmetric()
    .refined(3)
    .with_boundaries(
        {
            "left": lambda x: x[0] == 0,
            "right": lambda x: x[0] == 1,
            "top": lambda x: x[1] == 1,
        }
    )
)

e = ElementTriMorley()
ib = Basis(m, e)


@BilinearForm
def bilinf(u, v, w):
    from skfem.helpers import dd, ddot, trace, eye
    d = 0.1
    E = 200e9
    nu = 0.3

    def C(T):
        return E / (1 + nu) * (T + nu / (1 - nu) * eye(trace(T), 2))

    return d**3 / 12.0 * ddot(C(dd(u)), dd(v))


K = asm(bilinf, ib)
f = 1e6 * asm(unit_load, ib)

D = np.hstack([ib.get_dofs("left"), ib.get_dofs({"right", "top"}).all("u")])


A,b,temp1,temp2=condense(K, f, D=D)

In [3]:
A

<512x512 sparse matrix of type '<class 'numpy.float64'>'
	with 5454 stored elements in Compressed Sparse Row format>

In [4]:
Problem='Ex11'
#Linear elasticity equation
r"""Linear elasticity.

This example solves the linear elasticity problem using trilinear elements.  The
weak form of the linear elasticity problem is defined in
:func:`skfem.models.elasticity.linear_elasticity`.

"""

import numpy as np
from skfem import *
from skfem.models.elasticity import linear_elasticity, lame_parameters

m = MeshHex().refined(3)
e1 = ElementHex1()
e = ElementVector(e1)
ib = Basis(m, e, MappingIsoparametric(m, e1), 3)

K = asm(linear_elasticity(*lame_parameters(1e3, 0.3)), ib)

dofs = {
    'left' : ib.get_dofs(lambda x: x[0] == 0.0),
    'right': ib.get_dofs(lambda x: x[0] == 1.0),
}

u = ib.zeros()
u[dofs['right'].nodal['u^1']] = 0.3

I = ib.complement_dofs(dofs)

u = solve(*condense(K, x=u, I=I))

A,b,temp1,temp2=condense(K, x=u, I=I)

In [5]:
A

<1701x1701 sparse matrix of type '<class 'numpy.float64'>'
	with 106875 stored elements in Compressed Sparse Row format>

In [6]:
Problem='Ex20'
#Creeping flow problem
r"""Creeping flow.

The stream-function :math:`\psi` for two-dimensional creeping flow is
governed by the biharmonic equation

.. math::
    \nu \Delta^2\psi = \mathrm{rot}\,\boldsymbol{f}
where :math:`\nu` is the kinematic viscosity (assumed constant),
:math:`\boldsymbol{f}` the volumetric body-force, and :math:`\mathrm{rot}\,\boldsymbol{f} \equiv
\partial f_y/\partial x - \partial f_x/\partial y`.  The boundary
conditions at a wall are that :math:`\psi` is constant (the wall is
impermeable) and that the normal component of its gradient vanishes (no
slip).  Thus, the boundary value problem is analogous to that of
bending a clamped plate, and may be treated with Morley elements as in
the Kirchhoff plate tutorial.

Here we consider a buoyancy force :math:`\boldsymbol{f} = x\hat{j}`,
which arises in the Boussinesq approximation of natural convection
with a horizontal temperature gradient (`Batchelor 1954
<http://dx.doi.org/10.1090/qam/64563>`_).

For a circular cavity of radius :math:`a`, the problem admits a
polynomial solution with circular stream-lines:

.. math::
    \psi = \left(1 - (x^2+y^2)/a^2\right)^2 / 64.

"""

from skfem import *
from skfem.models.poisson import unit_load
from skfem.models.general import curluv
from skfem.helpers import ddot, dd

import numpy as np


mesh = MeshTri.init_circle(4)
element = ElementTriMorley()
mapping = MappingAffine(mesh)
ib = Basis(mesh, element, mapping, 2)


@BilinearForm
def biharmonic(u, v, w):
    return ddot(dd(u), dd(v))


stokes = asm(biharmonic, ib)
rotf = asm(unit_load, ib)

A,b,temp1,temp2=condense(stokes, rotf, D=ib.get_dofs())

In [7]:
A

<1985x1985 sparse matrix of type '<class 'numpy.float64'>'
	with 21997 stored elements in Compressed Sparse Row format>

In [8]:
Problem='Ex38'
#Greens function for a disk
r"""Point source.

Sources concentrated at points cannot be evaluated in the usual way, which
involves discrete quadrature; instead, it requires direct use of the basis
functions, as implemented in `CellBasis.point_source`.

Here this is demonstrated for a disk with homogeneous Dirichlet conditions.
The exact solution is the well-known Green's function (e.g. Sadybekov,
Turmetov, & Torebek 2015).

* Sadybekov, M. A., Turmetov, B. K. & Torebek, B. T. (2015). On an explicit
  form of the Green function of the Robin problem for the Laplace operator
  in a circle. *Advances in Pure and Applied Mathematics,* 6, 163-172.
  [doi: 10.1515/apam-2015-0003](https://doi.org/10.1515%2fapam-2015-0003)



"""
from functools import partial
from pathlib import Path

from skfem import *
from skfem.models.poisson import laplace, mass, unit_load
from skfem.io.json import from_file

import numpy as np


def greens(a: float, s: np.ndarray, x: np.ndarray) -> np.ndarray:
    """Return the Green's function for a disk of radius `a`

    with source at point `s`, evaluated at points `x`.
    """

    snorm = np.linalg.norm(s)
    sfull = s[:, None, None]
    numerator = np.linalg.norm(snorm ** 2 * x - a ** 2 * sfull, axis=0)
    denominator = a * snorm * np.linalg.norm(x - sfull, axis=0)
    return np.log(numerator / denominator) / 2 / np.pi


basis = Basis(MeshTri.init_circle(5), ElementTriP2())
source = np.array([0.3, 0.2])

A = asm(laplace, basis)
b = basis.point_source(source)

A,b,temp1,temp2=condense(A, b, D=basis.get_dofs())

In [9]:
A

<8065x8065 sparse matrix of type '<class 'numpy.float64'>'
	with 90201 stored elements in Compressed Sparse Row format>

In [85]:
def aktuell(x):
    #print(x)
    print(scipy.linalg.norm(x))
    timestop=time.time()
    #print(scipy.linalg.norm(bnonspd-Anonspd.dot(x)))
    with open(f'{Problem}.csv', 'a') as file:
        file.write(f",{scipy.linalg.norm(x)},{timestop-timestart}")

In [86]:
N=A.shape[0]


In [3]:
for fill_factor in range(1,10):
    for drop_tolerance in range(-12,5,2):
        print(fill_factor, drop_tolerance)
            
        with open(f'{Problem}.csv', 'a') as file:
            file.write(f"{fill_factor},{drop_tolerance}")
        timestart = time.time()
        
        # ILUT Variante aus scipy
        ilu = spilu(A, fill_factor=fill_factor, drop_tol=pow(10,drop_tolerance))
        Mx = lambda x: ilu.solve(x)
        M = LinearOperator((N, N), Mx)
        with open(f'{Problem}.csv', 'a') as file:
            file.write(f",{ilu.nnz}")
        
        #in the experiments we use an absolute stopping criterion,
        #which means setting the relative tolerance tol too big for it to effect termination
        #the absolute stopping criterion depends heavily on the linear system
        x, exit_code = cg(A, b,callback=aktuell, M=M, callback_type='r',tol=1e-10, atol=1e-16, maxiter=500)
        # if nonspd:
        #x, exit_code = gmres(A, b,callback=aktuell, M=M, callback_type='pr_norm',tol=1e-100, atol=1e-11, maxiter=500)
        print(exit_code)    # 0 indicates successful convergence
        print('scipy.linalg.norm(b-A.dot(x)): ',scipy.linalg.norm(b-A.dot(x)))
        with open(f'{Problem}.csv', 'a') as file:
            file.write("\n")
            

1 -12


NameError: name 'Problem' is not defined

In [ ]:
scipy.linalg.norm(b-A.dot(x))

In [ ]:
np.power(2,4)

In [14]:
pow(4, 3)

64

In [15]:
pow(2,-1)

0.5